In [93]:
import pandas as pd
import numpy as np
import sqlite3
import urllib
import datetime
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
import pickle

In [83]:
class 크롤러:
    def __init__(self, driver):
        self.driver = driver
        self.date = str(datetime.date.today())
    def __홈페이지접근(self, url, params):
        url_params = '{}?{}'.format(url, urllib.parse.urlencode(params))
        if self.driver.current_url != url_params:
            self.driver.get(url_params)
    def 상품코드수집(self):
        url = 'https://www.meatbox.co.kr/fo/product/productListPage.do'
        params = {'categorySeq':10005,'articleCount':'1000'}
        self.__홈페이지접근(url, params)
        상품코드 = self.driver.find_elements_by_css_selector('li[data-product-seq]')
        상품코드정보 = []
        for 상품 in 상품코드:
            상품코드정보.append((self.date, 상품.get_attribute('data-product-seq')))
        상품코드정보 = pd.DataFrame(상품코드정보, columns=['기준일자','상품코드'])
        return 상품코드정보
    def 추가정보수집(self, 상품코드):
        url = 'https://www.meatbox.co.kr/fo/product/productViewPage.do'
        params = {'productSeq':상품코드}
        self.__홈페이지접근(url, params)
        
        # 상세정보수집
        테이블 = driver.find_element_by_xpath("//table[@class='section_td']").find_elements_by_tag_name('tr')
        상세정보 = {'상품코드':상품코드}
        for 레코드 in 테이블:
            속성 = 레코드.find_element_by_tag_name('th').text
            내용 = 레코드.find_element_by_tag_name('td').text
            상세정보[속성] = 내용
            
        # 판매정보수집
        테이블 = driver.find_element_by_xpath("//li[@class='bott_info']").find_elements_by_tag_name('dl')
        판매정보 = {'기준일자':self.date, '상품코드':상품코드}
        for 레코드 in 테이블:
            속성 = 레코드.find_element_by_tag_name('dt').get_attribute('innerHTML')
            내용 = 레코드.find_element_by_tag_name('em').get_attribute('innerHTML')
            판매정보[속성] = 내용

        # 판매자정보수집
        테이블 = driver.find_element_by_xpath("//div[@class='pop_comm sale_pop']").find_elements_by_tag_name('tr')
        판매자정보 = {'상품코드':상품코드}
        for 레코드 in 테이블:
            속성 = 레코드.find_element_by_tag_name('th').get_attribute('innerHTML').replace('&nbsp;', ' ')
            내용 = 레코드.find_element_by_tag_name('td').get_attribute('innerHTML').replace('&nbsp;', ' ')
            판매자정보[속성] = 내용
        판매자정보['사업자등록번호'] = 판매자정보['사업자등록번호'].split('<strong>')[0]

        # 상품정보수집
        try:
            테이블 = driver.find_element_by_xpath("//div[@class='pop_comm deteil_over_pop _optionDetailInfoLayer']").find_element_by_xpath("//div[@class='table_wrap']").find_elements_by_tag_name('tr')
            상품정보 = {'상품코드':상품코드}
            for 레코드 in 테이블:
                속성 = 레코드.find_element_by_tag_name('th').get_attribute('innerHTML')
                내용 = 레코드.find_element_by_tag_name('td').get_attribute('innerHTML')
                상품정보[속성] = 내용
        except NoSuchElementException:
            상품정보 = {'상품코드':상품코드}

        return (상세정보, 판매정보, 판매자정보, 상품정보)

In [85]:
driver = webdriver.Chrome('./downloads/chromedriver')
crawler = 크롤러(driver)
상품코드 = crawler.상품코드수집()

In [90]:
상세정보들, 판매정보들, 판매자정보들, 상품정보들 = [], [], [], []
for 코드 in 상품코드['상품코드']:
    상세정보, 판매정보, 판매자정보, 상품정보 = crawler.추가정보수집(코드)
    상세정보들.append(상세정보)
    판매정보들.append(판매정보)
    판매자정보들.append(판매자정보)
    상품정보들.append(상품정보)

In [109]:
정보목록 = ['상세정보', '판매정보', '판매자정보', '상품정보']
for 정보 in 정보목록:
    with open(정보+'.obj', 'wb') as f:
        pickle.dump(상세정보들, f)